In [ ]:
import os
from pathlib import Path
from pdf2image import convert_from_path

from PIL import Image 
import io
from google.cloud import vision

poppler = "/project/arcc-students/csloan5/environments/GPU_env/bin/"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'savvy-hybrid-389017-984cabbf64b0.json'

def pdf_to_jpg(pdf_path, buf):
    pages = convert_from_path(pdf_path, poppler_path=poppler)
    page = pages[0] 
    buf = io.BytesIO()
    page.save(buf, 'JPEG')


    client = vision.ImageAnnotatorClient()

    with io.open(jpg_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content) 
    response = client.document_text_detection(image=image)

    text = response.full_text_annotation.text
    return text

pdf_path = 'TheToughOne.pdf'
pdf_to_jpg(pdf_path, buf)

text = ocr_image(

with open('output.txt', 'w') as f:
    f.write(text)


In [ ]:
import io
import os
import time
import tempfile
from pdf2image import convert_from_path
from google.cloud import vision_v1p3beta1 as vision

# Path to the PDF document
pdf_path = 'TheToughOne.pdf'

# Path to the authentication JSON file
auth_json_path = 'savvy-hybrid-389017-984cabbf64b0.json'

# Set up Google Cloud Vision API client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = auth_json_path
client = vision.ImageAnnotatorClient()
path_to_poppler = r'/project/arcc-students/csloan5/environments/GPU_env/bin/'

with tempfile.TemporaryDirectory() as temp_dir:
    images = convert_from_path(
        pdf_path,
        dpi=300,
        poppler_path=path_to_poppler,
        output_folder=temp_dir,
        fmt='jpeg'
    )

    for i, image in enumerate(images):
        # Save the JPEG image
        image_path = os.path.join(temp_dir, f'page_{i+1}.jpeg')
        image.save(image_path)

        # Read the image file as bytes
        with io.open(image_path, 'rb') as img_file:
            image_content = img_file.read()

        # Create an image object
        vision_image = vision.Image(content=image_content)

        # Set the image context for handwriting recognition
        image_context = vision.ImageContext(language_hints=['en-t-i0-handwrit'])

        # Perform asynchronous handwritten OCR on the image
        response = client.document_text_detection(image=vision_image, image_context=image_context)

        # Get the extracted text
        if response.text_annotations:
            extracted_text = response.text_annotations[0].description
            print(f'Extracted Text from Page {i+1}:')
            print(extracted_text)
        else:
            print(f'No handwritten text found on Page {i+1}.')

In [ ]:
import io
import os
import json
from google.cloud import vision


# Set up Google Cloud Vision API client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'savvy-hybrid-389017-984cabbf64b0.json'
client = vision.ImageAnnotatorClient()
from google.protobuf.json_format import MessageToJson
from PIL import Image
from pdf2image import convert_from_path
import io
import numpy as np
import cv2
poppler = "/project/arcc-students/csloan5/environments/GPU_env/bin/"
pdf_path = "Slides/145-0288.pdf"

image = convert_from_path(pdf_path, poppler_path=poppler)
image = image[0]
#image.save('Slides/334-0173.pdf_page_5.jpg', 'JPEG')
buf = io.BytesIO()
image.save(buf, format='JPEG')
byte_im = buf.getvalue()

# Convert the PDF content to an image
image = vision.Image(content=byte_im)

# Get full response
# response = client.document_text_detection(image=image)

# # Get the full text annotation object
# full_text = response.full_text_annotation

# Serialize to JSON string
# json_str = MessageToJson(full_text, descriptor_pool=None)

response = client.document_text_detection(image=image)
json_string = type(response).to_json(response)  # Also works to use the message type directly, e.g.
                                                # json_string = MyMessageType.to_json(response)

# Write JSON string to file
with open('output.json', 'w') as f:
    f.write(json_string)

print('Full text annotation saved to output.json')

In [ ]:
pip install proto-plus

In [ ]:
import io
import os
from google.cloud import vision
from pdf2image import convert_from_path

poppler = "/project/arcc-students/csloan5/environments/GPU_env/bin/"

# Set up Vision client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'savvy-hybrid-389017-984cabbf64b0.json' 
client = vision.ImageAnnotatorClient()

# Folder containing PDFs
folder_path = '/project/arcc-students/OilWellProject2023.Summer/vert_pages'
output_folder = 'jsons/Testing/Vert_Jsons/'
text_check_folder = 'jsons'

# Loop through all PDFs in folder
for files in os.walk(folder_path):
    print(files)
    break
    for pdf_path in files:
        
        print(pdf_path)
        break
        if pdf_path.endswith('.pdf'):
            
            # Get filename without extension
            filename = os.path.splitext(pdf_path)[0]

#             # Check text file folder for existing text file
#             text_path = os.path.join(text_check_folder, f"{filename}.json")
#             if os.path.exists(text_path):
#                 print(f"{text_path} exists, skipping OCR")
#                 continue

            # Convert PDF to image
            images = convert_from_path(os.path.join(folder_path, pdf_path),poppler_path=poppler)
            image = images[0]

            # Convert image to bytes (what google cloud vision uses as their input)
            buf = io.BytesIO()
            image.save(buf, format='JPEG')
            byte_im = buf.getvalue()

            # Perform OCR
            image = vision.Image(content=byte_im)
            response = client.document_text_detection(image=image, image_context={"language_hints": ['en']})

        #         # Extract text
        #         extracted_text = response.full_text_annotation.text
            response = client.document_text_detection(image=image)
            json_string = type(response).to_json(response)        
            # Save text to output folder
            text_path = os.path.splitext(pdf_path)[0] + '.json'
            output_path = os.path.join(output_folder, text_path)


            with open(output_path, 'w') as text_file:
                print(json_string, file=text_file)
            
print('OCR complete for all PDFs in folder')

In [ ]:
import os

folder = 'BlankPages/VertNonBlank'

# Get list of all files in the folder
files = os.listdir(folder)

# Track file names in a set
file_names = set()

for file in files:
    name_ext = os.path.splitext(file)
    # If filename already in set, it's a duplicate
    if name_ext[0] in file_names:
        print('bruh')
        new_name = name_ext[0] + '2' + name_ext[1]
        os.rename(os.path.join(folder, file), os.path.join(folder, new_name))
  
    # Add original name to set
    else:
        file_names.add(file)
    
print('Renamed duplicate files in folder by adding 2')

In [ ]:
file = 'hello/bruh.jpeg'

name_ext = os.path.splitext(file)

print(name_ext[0])

In [ ]:
pip install duplicate-finder

In [ ]:
import duplicates as dup


folder_of_interes = 'BlankPages/VertNonBlank'

dir = 'dupes.csv'
if not os.path.exists(dir):
    os.mkdir(dir)
dup.list_all_duplicates(folder_of_interest, to_csv=True, csv_path=dir, fastscan=False)


In [5]:
import io
import os
from google.cloud import vision
from pdf2image import convert_from_path

poppler = "/project/arcc-students/csloan5/environments/GPU_env/bin/"

# Set up Vision client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'savvy-hybrid-389017-984cabbf64b0.json' 
client = vision.ImageAnnotatorClient()

# Folder containing PDFs 
pdf_folder = '/project/arcc-students/OilWellProject2023.Summer/vert_pages/Box14/'
output_folder = 'jsons/Testing/Vert_Jsons/'

# Loop through all PDFs in folder
for pdf_path in os.listdir(pdf_folder):
    if pdf_path.endswith('.pdf'):

        # Get filename without extension
        filename = os.path.splitext(pdf_path)[0]

        # Convert PDF to image
        images = convert_from_path(os.path.join(pdf_folder, pdf_path),poppler_path=poppler)
        image = images[0]

        # Convert image to bytes
        buf = io.BytesIO()
        image.save(buf, format='JPEG')
        byte_im = buf.getvalue()

        # Perform OCR
        image = vision.Image(content=byte_im)
        response = client.document_text_detection(image=image, image_context={"language_hints": ['en']})

        # Extract text as JSON
        json_string = type(response).to_json(response) 

        # Save text to output folder
        output_path = os.path.join(output_folder, f"{filename}.json")
        with open(output_path, 'w') as outfile:
            outfile.write(json_string)

print('OCR complete for all PDFs in folder')

OCR complete for all PDFs in folder
